In [1]:
from random import choice, choices
import base64
from Crypto.Cipher import AES

BLOCKSIZE = 16

def Pad(s: bytes):
    assert type(s) is bytes
    npad = BLOCKSIZE - len(s) % BLOCKSIZE
    return s + bytes( [npad] * npad )

def Unpad(s: bytes):
    assert type(s) is bytes and len(s) % BLOCKSIZE == 0
    last = s[-1]
    assert 1 <= last <= 16 and all( byte == last for byte in s[-last:] )
    return s[:-last]

with open('strings.txt', mode = 'rb') as file:
    strings = [ base64.decodebytes(line) for line in file ]

def Chunkerize(x, chunksize, strict = True):
    x = list(x)
    assert len(x) % chunksize == 0 if strict else True
    for n in range( len(x) // chunksize ):
        yield x[ n*chunksize : (n+1)*chunksize ]

In [2]:
def PadPlaintext(plaintext: bytes, blocksize = 16):
    npad = blocksize - len(plaintext) % blocksize
    return plaintext + bytes([npad]) * npad

def XOR(X: bytes, Y: bytes) -> bytes:
    assert type(X) is bytes and type(Y) is bytes and len(X) == len(Y)
    return bytes([ x^y for x, y in zip(X, Y) ])

def EncryptCBC(plaintext, key, initialization):
    BLOCKSIZE = 16
    plaintext = PadPlaintext(plaintext, BLOCKSIZE)
    assert type(initialization) is bytes and len(initialization) == BLOCKSIZE
    ECBcipher = AES.new(key, AES.MODE_ECB)
    
    plain_blocks = [ bytes(block) for block in Chunkerize(plaintext, BLOCKSIZE) ]
    cipher_blocks = [None] * len(plain_blocks)
    
    for n in range(len(plain_blocks)):
        if n == 0:
            plain_block = XOR(plain_blocks[n], initialization)
            cipher_blocks[n] = ECBcipher.encrypt(plain_block)
        else:
            plain_block = XOR(plain_blocks[n], cipher_blocks[n-1])
            cipher_blocks[n] = ECBcipher.encrypt(plain_block)
    
    return b''.join(cipher_blocks)

def DecryptCBC(ciphertext, key, initialization):
    BLOCKSIZE = 16
    assert len(ciphertext) % BLOCKSIZE == 0
    assert type(initialization) is bytes and len(initialization) == BLOCKSIZE
    ECBcipher = AES.new(key, AES.MODE_ECB)
    
    cipher_blocks = [ bytes(block) for block in Chunkerize(ciphertext, BLOCKSIZE) ]
    plain_blocks = [None] * len(cipher_blocks)
    
    for n in range(len(plain_blocks)):
        if n == 0:
            plain_block = ECBcipher.decrypt(cipher_blocks[n])
            plain_blocks[n] = XOR(plain_block, initialization)
        else:
            plain_block = ECBcipher.decrypt(cipher_blocks[n])
            plain_blocks[n] = XOR(plain_block, cipher_blocks[n-1])
            
    plaintext = b''.join(plain_blocks)
    return plaintext
 

In [22]:
key = b'\x13c\xb1I\x17\xe7dj\xd5?\xbe\xb9f\x03\xa1\x84'
strings = [ base64.decodebytes(line) for line in open('strings.txt', mode = 'rb') ]


def Function1():    
    initialization = bytes( choices(range(256), k = 16) )
    
    plaintext = choice(strings)
    ciphertext = EncryptCBC(plaintext, key, initialization)
    return initialization, ciphertext

def Function2(initialization, ciphertext):
    try:
        plaintext = DecryptCBC(ciphertext, key, initialization)
        plaintext = Unpad(plaintext)
        return True
    except:
        return False

In [23]:
# First I'll decrypt the last byte just to try

initialization, ciphertext = Function1()

In [24]:
plaintext = b'A'*(BLOCKSIZE+1)
initialization = b'iiiiiiiiiiiiiiii'
ciphertext = EncryptCBC(plaintext, key, initialization)


In [6]:
DecryptCBC(ciphertext, key, initialization)

b'AAAAAAAAAAAAAAAAA\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f'

In [7]:
# Solve the last char to get a feel

new_ciphertext = list(ciphertext)

for char in range(256):
    if char == ciphertext[15]:
        continue
    new_ciphertext[15] = char
    if Function2(initialization, bytes(new_ciphertext)):
        print(char^1^ciphertext[15])

15


In [8]:
# Ok so last char is 15
# let's do second to last

last_char = 15

new_ciphertext = list(ciphertext)

new_ciphertext[15] = 2 ^ last_char ^ ciphertext[15]
new_ciphertext[14] = 2 ^ 15 ^ ciphertext[14]

DecryptCBC(bytes(new_ciphertext), key, initialization)
Function2(initialization, bytes(new_ciphertext[:]))

True

In [15]:
def SolveBlockPair(block1, block2):
    assert type(block1) is type(block2) is bytes
    assert len(block1) == len(block2) == BLOCKSIZE
    
    # Find the last block
    pad = 1
    possible_solutions = list()
    new_ciphertext = list(block1)
    for char in range(0xFF): 
        if char == block1[BLOCKSIZE-1]:
            continue
        new_ciphertext[BLOCKSIZE-1] = char
        if Function2(bytes(new_ciphertext), block2):
            possible_solutions += [ char^pad^block1[BLOCKSIZE-1] ]
            
    assert len(possible_solutions) == 1
    known_chars = [ *possible_solutions ]
    
    # Now work on the others
    for pad in range(2, BLOCKSIZE+1):
        assert len(known_chars) == pad-1
        idx = BLOCKSIZE-pad # index of char we are currently trying to solve for
        new_ciphertext = list(block1)
        assert len(known_chars) == len(block1[idx+1:])
        new_ciphertext[idx+1:] = [ pad^known_char^cipher_char for known_char, cipher_char in zip(known_chars, block1[idx+1:]) ]
        possible_solutions = list()
        
        for char in range(0xFF):
            new_ciphertext[idx] = char
            decrypted = DecryptCBC(block2, key, bytes(new_ciphertext))
            assert len(decrypted) == BLOCKSIZE
            assert all( item == pad for item in decrypted[idx+1:] )
            if decrypted[idx] == pad:
                possible_solutions += [ char ]
        
        assert len(possible_solutions) == 1
        char, = possible_solutions
        known_chars = [ char^pad^block1[idx], *known_chars ]

    return known_chars
    
def 

In [16]:
SolveBlockPair(ciphertext[:16], ciphertext[16:])

[65, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]

In [11]:
14^15

1

In [12]:
x = 257
y = 257
x is y

False